In [13]:
import cv2
import os
from google.cloud import storage
from datetime import datetime

In [14]:
# GCS Ayarları
GCS_CREDENTIALS = r"C:\Users\enesb\Documents\GitHub\Real-Time-Face-Recognition-and-Attendance-System\real-time-attendance-460605-4367d4b382a9.json"
BUCKET_NAME = "dataset-aee"

In [15]:
# GCS Bağlantısı Kur
storage_client = storage.Client.from_service_account_json(GCS_CREDENTIALS)
bucket = storage_client.bucket(BUCKET_NAME)

In [16]:
# Kullanıcıdan bilgi al
fakulte = input("Fakülte adı (örn: Mühendislik ve Doğa Bilimleri Fakültesi): ").strip()
bolum = input("Bölüm adı (örn: Yazılım Mühendisliği): ").strip()
sinif = input("Sınıf (örn: 1. Sınıf): ").strip()
isim = input("Ad Soyad (örn: Ali Yılmaz): ").strip()
isim_kodu = isim.replace(" ", "_")

In [17]:
# Kayıt yolu oluştur
local_path = os.path.join("dataset", fakulte, bolum, sinif, isim_kodu)
os.makedirs(local_path, exist_ok=True)
print(f"Yerel klasör oluşturuldu: {local_path}")

Yerel klasör oluşturuldu: dataset\Hukuk Fakültesi\Hukuk\3. Sınıf\Furkan_Dağ


In [18]:
import cv2
import numpy as np
import io

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Kamera açılmadı!")
    exit()

print("Fotoğraf çekmek için hazırsın. 5 fotoğraf çekilecek...")
foto_sayisi = 5
cekilen = 0

while cekilen < foto_sayisi:
    ret, frame = cap.read()
    if not ret:
        continue

    cv2.imshow("Yüz Kaydı - 's' ile kaydet, 'q' ile çıkış", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('s'):
        filename = f"{cekilen+1}.jpg"
        gcs_blob_path = f"dataset/{fakulte}/{bolum}/{sinif}/{isim_kodu}/{filename}"
        blob = bucket.blob(gcs_blob_path)

        # Belleğe encode edip upload et
        success, encoded_image = cv2.imencode('.jpg', frame)
        if success:
            image_bytes = encoded_image.tobytes()
            blob.upload_from_string(image_bytes, content_type='image/jpeg')
            print(f"GCS'ye doğrudan yüklendi: {gcs_blob_path}")
            cekilen += 1
        else:
            print("Fotoğraf encode edilemedi.")

    elif key == ord('q'):
        break

print("Kayıt tamamlandı. Kamera kapanıyor...")
cap.release()
cv2.destroyAllWindows()


Fotoğraf çekmek için hazırsın. 5 fotoğraf çekilecek...
GCS'ye doğrudan yüklendi: dataset/Hukuk Fakültesi/Hukuk/3. Sınıf/Furkan_Dağ/1.jpg
GCS'ye doğrudan yüklendi: dataset/Hukuk Fakültesi/Hukuk/3. Sınıf/Furkan_Dağ/2.jpg
GCS'ye doğrudan yüklendi: dataset/Hukuk Fakültesi/Hukuk/3. Sınıf/Furkan_Dağ/3.jpg
GCS'ye doğrudan yüklendi: dataset/Hukuk Fakültesi/Hukuk/3. Sınıf/Furkan_Dağ/4.jpg
GCS'ye doğrudan yüklendi: dataset/Hukuk Fakültesi/Hukuk/3. Sınıf/Furkan_Dağ/5.jpg
Kayıt tamamlandı. Kamera kapanıyor...
